In [8]:
from mido import Message
from mido import MidiFile
from pymidifile import *
from reformat_midi import *
import numpy as np
import os

In [21]:
def GetTrackName(track):
    """Extract the track name in a track.
    """
    for msg in track:
        if (msg.type == 'track_name'):
            return msg.name
    
    
def FromMIDIToChromosome(filename):
    """Convert a MIDI file into chromosome representation
    Parameters:
        filename: Path of the MIDI file. The file should be single-track and properly quantized.
    
    Return value:
        a list of numbers in the range 0..28, viz. the chromosome representation. 
        (Pitches beyond the range 1..27 are discarded.)
    [Note] Integer representation of pitches in Chromosome:
        0 = rest,
        1 = F3, 2 = #F3, ... , 26 = #F5, 27 = G5
        28 = duration extension
        unit duration = an eighth note
    [Note] Integer representation of pitches as MIDI note number:
        60 = C4
        53 = F3, 54 = #F3, ... , 78 = #F5, 79 = G5
    [Note] Rule of convertion:
        Chromosome number + 52 == MIDI note number
    
    Notes
    """
    
    #mid = MidiFile(filename)
    #print(mid.ticks_per_beat)
    #for i,track in enumerate(mid.tracks):
    #    print(i,GetTrackName(track))
    #x = int(input())
    #for msg in mid.tracks[0]:
    #    print(msg)
    reformatted = reformat_midi(filename, verbose=False, write_to_file=False, override_time_info=True)
    matrix = mid_to_matrix(reformatted)
    quantizer = quantize_matrix(matrix, stepSize=0.5, quantizeOffsets=True, quantizeDurations=True)
    note_list = np.array(sorted(quantizer, key=lambda x: x[0], reverse=False))
    #print(note_list[:,1]) # unit duration = quarter note
    #print(note_list[:,2])
    #print(note_list[:,1]+note_list[:,2])
    #print(2*np.max(note_list[:,1]+note_list[:,2]))
    length = int(2*np.max(note_list[:,1]+note_list[:,2]))
    output = np.zeros(length)
    #for note in note_list:
    for note in sorted(quantizer, key=lambda x: x[0], reverse=False):
        #print(note)
        start = int(2*note[1])
        dur = int(2*note[2])
        pitch = int(note[0] - 52)
        if (pitch in range(1,28)): # if pitch < 1 (F3) or pitch > 27 (G5), it is neglected (replaced with rests)
            for t in range(start, start+dur):
                output[t] = pitch
    #for i in range(length-1, 0, -1):
        #if (output[i] == output[i-1]):
            #output[i] = 28
    output_list = output.tolist()
    #print(output.tolist())
    while (len(output_list) >= 32):
        for i in range(31, 0, -1):
            if (output_list[i] == output_list[i-1]):
                output_list[i] = 28
        #print(output_list[:32])
        print(0, end = ' ')
        for i in range(0, 32):
            print(int(output_list[i]), end = ' ')
        print('')
        output_list = output_list[32:]
    #return output.tolist()

In [9]:
def FromMIDIFolderToChromosome(foldername):
    for root, dirs, files in os.walk(foldername):
        for name in files:
            FromMIDIToChromosome(os.path.join(root, name))

In [ ]:
FromMIDIToChromosome('chopin.mid')

In [22]:
FromMIDIFolderToChromosome('./data/chopin')

file name: ./data/chopin/nocturne_62_2_(c)unknown.mid
NAME nocturne_62_2_(c)unknown.mid
78
0 0 28 28 28 28 28 7 28 16 28 28 17 19 23 24 21 19 28 28 28 14 28 28 16 12 28 28 28 14 16 17 19 
0 21 28 28 28 0 21 28 19 28 28 17 28 0 16 28 14 13 28 14 28 0 6 7 17 16 28 11 12 11 12 28 14 
0 12 28 28 28 11 23 21 19 16 28 28 17 24 26 28 21 28 19 28 28 14 28 28 16 12 28 11 12 14 16 17 19 
0 21 28 28 28 0 21 28 19 21 19 18 28 28 19 21 22 23 28 28 21 18 19 9 0 28 28 14 28 28 19 23 28 
0 18 28 28 28 19 28 0 28 16 28 28 17 19 21 24 22 13 28 28 28 14 28 0 28 18 28 28 19 21 23 26 24 
0 15 28 28 28 16 17 21 19 13 28 28 28 14 15 19 17 28 28 15 14 11 28 28 28 27 28 24 20 8 5 15 6 
0 7 28 28 28 9 11 12 14 16 28 28 17 26 0 28 26 21 19 28 28 14 28 28 15 12 28 28 13 15 19 20 17 
0 15 28 28 28 10 28 28 12 9 28 28 28 28 11 12 13 14 28 7 28 16 28 28 28 0 16 21 26 28 16 28 0 
0 12 28 0 28 28 28 12 28 16 28 28 28 28 28 12 14 16 28 28 28 14 28 12 28 21 28 19 28 12 28 14 16 
0 17 16 0 28 28 12 24 28 28 28 26 24 16 1

72
0 0 28 28 28 28 28 28 28 28 22 28 28 28 20 18 20 17 0 28 28 18 28 17 14 28 28 28 15 28 17 18 20 
0 20 22 23 17 28 28 28 28 27 28 25 22 10 22 20 18 17 0 6 18 28 20 17 14 28 0 28 15 28 17 18 20 
0 20 22 28 15 28 28 28 17 18 28 28 28 28 19 28 0 20 28 21 22 23 27 28 25 20 28 21 22 28 23 28 0 
0 0 24 28 25 27 25 0 28 23 28 24 25 28 27 28 23 28 18 20 22 28 25 22 20 28 15 17 18 28 17 18 20 
0 22 20 25 23 22 28 28 28 28 28 28 28 28 20 28 28 17 0 28 28 18 20 17 14 28 0 28 15 28 17 18 20 
0 22 28 23 17 28 28 28 28 27 28 25 22 10 22 20 18 17 0 6 18 28 20 17 14 28 0 28 15 28 17 18 20 
0 20 22 28 15 28 28 28 17 18 28 28 28 28 19 28 0 20 28 21 22 23 27 28 25 20 28 21 22 28 23 28 0 
0 0 24 28 25 27 25 0 28 23 28 24 25 28 27 28 23 28 18 20 22 28 25 22 20 28 15 17 18 28 17 18 20 
0 22 20 25 23 22 28 28 28 28 28 28 28 28 20 28 28 17 0 28 28 18 20 17 14 28 0 28 15 28 17 18 20 
0 20 22 23 28 17 0 28 27 26 20 18 22 10 22 20 18 17 0 6 18 28 28 17 14 28 28 28 15 28 17 18 20 
0 20 22 28 15 28 28 28 6 28 28

465
0 0 28 28 28 28 28 28 28 13 28 28 28 28 9 0 28 13 28 4 13 28 9 4 25 28 28 28 28 28 28 28 23 
0 23 28 21 28 28 28 28 13 16 28 28 28 28 28 28 28 28 21 28 25 28 28 0 28 13 4 13 9 1 0 28 13 
0 13 24 25 20 21 0 17 28 28 28 28 28 28 28 28 28 28 28 0 18 28 28 28 28 28 28 28 0 28 14 28 6 
0 25 14 11 26 25 28 28 28 28 23 28 28 28 25 23 25 28 26 28 2 0 26 0 28 14 5 14 28 0 26 20 28 
0 21 22 23 14 25 26 0 27 28 28 28 28 28 28 0 28 25 28 28 28 28 28 28 9 0 25 28 28 26 28 21 28 
0 23 28 20 25 28 28 28 23 28 28 21 18 21 13 20 26 25 28 28 28 28 28 28 28 28 28 26 28 23 21 25 13 
0 25 23 28 25 28 28 20 23 28 28 21 13 21 1 22 28 28 16 22 10 28 1 0 26 28 28 6 14 26 28 25 26 
0 26 28 0 25 28 28 26 28 28 25 23 22 18 20 21 23 10 24 26 28 0 26 28 6 14 11 26 16 25 16 26 28 
0 26 1 25 28 28 28 14 26 28 28 23 26 28 28 28 28 28 8 0 17 16 28 28 28 26 28 21 20 28 28 4 28 
0 16 28 28 17 28 19 12 16 17 24 28 28 28 23 28 28 2 28 11 17 16 28 28 28 28 26 16 26 24 23 20 0 
0 4 16 28 28 17 19 17 16 17 24 28 28 28 23 

96
0 0 28 28 28 28 28 28 28 28 17 20 24 22 28 0 17 20 24 28 26 27 25 24 20 12 28 0 20 28 25 24 28 
0 0 20 28 25 24 22 25 27 26 22 19 28 28 0 17 28 18 28 0 28 20 28 21 26 28 25 23 26 18 28 28 0 
0 26 25 26 28 0 26 28 25 26 25 26 28 25 26 25 28 28 28 26 23 20 28 28 18 20 17 13 28 0 25 27 24 
0 20 28 0 19 20 22 24 22 25 27 25 22 24 0 28 19 20 22 24 22 25 27 25 22 24 0 28 19 20 22 17 0 
0 0 28 14 28 15 28 0 28 12 28 20 27 25 23 21 18 20 0 28 15 16 18 20 27 25 23 21 18 14 0 28 28 
0 13 16 12 0 28 28 11 14 15 28 28 28 28 17 15 28 7 15 28 17 15 28 13 15 28 17 15 28 12 15 28 24 
0 24 28 22 20 28 17 15 28 28 28 28 17 15 28 13 15 28 17 15 28 12 15 28 20 28 28 19 17 28 12 19 28 
0 19 28 28 21 19 28 17 19 28 21 19 28 16 19 28 16 17 28 26 24 28 21 19 28 28 28 28 21 19 28 17 19 
0 19 21 19 28 16 19 28 21 22 28 17 22 28 21 28 28 28 28 28 22 28 28 21 28 28 26 28 28 25 28 28 22 
0 22 28 21 28 28 22 23 28 28 28 28 24 28 28 23 28 28 19 21 28 27 28 28 26 28 28 25 28 28 24 25 28 
0 25 13 28 15 13 28 11 13 

148
0 0 28 28 28 28 28 28 28 28 28 28 28 28 28 28 28 27 28 28 28 28 28 26 24 23 28 28 28 28 28 24 20 
0 19 28 28 14 20 19 17 15 14 28 15 28 14 0 28 28 19 28 28 28 28 28 21 28 22 28 28 28 28 28 28 28 
0 21 28 28 28 28 28 28 28 19 28 11 28 12 28 14 28 27 28 28 28 28 28 26 24 23 28 28 28 28 26 28 27 
0 19 28 28 28 28 28 28 17 14 28 15 24 23 28 28 28 22 28 28 28 28 28 20 28 13 19 28 15 28 28 28 28 
0 17 28 28 28 28 28 28 28 19 28 28 28 28 28 21 28 22 28 28 28 21 28 28 28 23 28 28 28 24 28 19 28 
0 18 28 28 28 19 28 28 28 28 28 28 28 18 28 28 28 19 28 28 28 0 28 28 28 23 28 28 28 21 28 28 0 
0 19 28 28 28 28 28 20 28 21 28 28 23 24 26 27 28 28 28 26 28 28 28 28 28 23 28 28 28 21 28 28 19 
0 21 28 19 28 28 28 28 28 18 28 28 28 15 14 15 28 17 28 28 28 28 28 0 28 27 28 28 28 28 28 20 19 
0 24 23 28 28 28 26 28 27 0 28 28 28 28 28 28 28 23 0 28 27 19 28 0 28 10 12 19 28 28 28 28 26 
0 0 28 28 28 28 28 28 28 28 28 28 28 28 28 28 28 28 28 11 28 12 28 14 28 27 28 28 28 28 28 26 24 
0 23 28 28 28 0

136
0 0 28 28 28 28 28 28 28 28 28 28 28 28 28 28 28 28 28 22 23 28 28 28 28 28 28 28 0 28 28 27 28 
0 27 19 28 28 20 28 28 28 28 22 28 28 23 27 28 25 22 28 28 28 28 28 23 28 28 24 28 28 25 28 28 23 
0 23 22 20 28 28 22 28 28 23 28 28 28 28 28 18 28 28 20 28 28 22 28 28 20 28 18 17 28 28 28 28 28 
0 15 28 28 28 28 28 16 28 28 20 28 18 27 28 0 28 28 28 28 28 16 22 28 28 23 28 28 0 28 28 27 28 
0 27 19 28 28 20 28 28 22 28 28 23 28 28 15 20 27 28 22 28 28 28 28 23 28 28 28 28 11 21 28 28 28 
0 21 28 28 28 28 20 28 19 20 28 25 23 28 28 21 28 28 20 28 19 20 28 28 23 28 28 28 28 28 22 28 21 
0 22 28 27 25 28 28 23 28 28 22 28 21 22 28 28 0 28 28 28 28 28 28 28 28 28 28 28 28 28 28 28 28 
0 0 28 28 28 27 28 28 28 28 28 22 28 28 27 28 28 23 28 28 22 28 20 23 28 28 28 28 28 22 28 28 25 
0 25 23 22 28 28 17 28 22 18 28 28 17 28 15 27 28 28 28 28 28 28 28 28 22 28 23 25 28 28 27 28 22 
0 23 28 28 22 28 20 0 28 28 28 28 28 28 28 28 17 28 18 19 16 20 18 22 27 25 23 0 28 28 28 28 28 
0 0 23 28 22 2

0 24 28 23 0 27 16 19 10 28 22 0 21 28 23 0 24 28 28 28 18 28 11 20 28 23 0 21 6 0 19 28 2 
0 2 0 28 2 28 14 28 26 14 28 26 24 0 26 24 23 26 24 23 21 24 23 21 19 28 21 23 17 0 16 17 19 
0 21 23 24 27 16 28 28 28 28 28 0 4 0 28 28 2 28 28 0 2 28 12 24 21 19 16 12 9 4 2 11 14 
0 23 0 28 28 2 28 14 27 26 14 27 26 24 27 26 24 23 27 26 24 28 23 20 19 17 19 20 14 19 15 18 19 
0 21 23 24 26 27 15 28 28 28 28 28 28 0 28 28 28 2 28 28 0 2 28 12 24 21 19 16 12 9 2 0 23 
0 23 28 28 28 28 28 28 24 23 21 23 24 26 14 28 23 28 28 28 28 28 28 28 28 24 23 16 21 23 24 26 14 
0 19 0 23 28 28 0 28 28 28 28 28 11 28 28 0 28 11 14 12 2 11 28 28 28 28 28 28 0 28 11 28 7 
0 0 11 28 14 12 0 11 28 28 28 0 11 28 19 28 14 0 11 28 28 28 0 28 9 28 28 0 28 28 11 28 28 
file name: ./data/chopin/nocturne_c_sharp_minor_(c)inoue.mid
NAME nocturne_c_sharp_minor_(c)inoue.mid
82
0 0 28 28 28 28 28 28 28 21 28 0 19 17 28 0 16 15 28 16 28 0 28 28 28 21 28 0 19 17 28 0 16 
0 15 28 16 28 0 28 28 28 28 28 28 28 26 28 28 28 28 0

8
0 0 28 28 28 28 28 28 28 28 28 14 28 23 22 23 22 21 22 21 28 28 28 23 24 26 0 26 25 26 28 24 28 
0 24 28 28 23 21 19 21 23 21 23 26 24 28 28 28 28 28 16 18 21 19 23 18 28 28 28 14 21 25 24 23 24 
0 23 22 21 28 28 20 23 25 27 0 26 27 0 25 24 28 28 28 28 23 21 23 21 16 28 21 0 28 28 26 25 23 
0 21 23 28 16 28 21 0 28 28 26 25 23 25 23 28 16 28 21 24 28 28 28 28 28 0 16 19 21 0 23 19 28 
0 19 28 0 28 23 22 23 22 21 22 21 28 28 28 23 24 26 0 26 25 26 25 24 28 28 28 28 23 28 19 21 23 
0 21 23 26 24 28 28 28 28 28 18 24 27 24 28 21 18 28 28 14 21 23 25 23 22 24 22 21 28 28 28 28 24 
0 26 28 28 28 25 28 24 28 28 28 28 23 21 23 21 16 28 21 0 28 28 26 25 23 21 23 28 16 28 21 0 28 
0 0 26 25 23 25 23 28 16 28 21 24 28 28 28 28 28 0 16 19 21 0 23 19 28 28 28 18 19 21 28 28 26 
0 26 23 21 28 28 19 28 28 0 28 26 28 19 21 18 28 28 28 28 28 28 28 28 23 28 19 18 28 16 0 28 28 
0 26 25 23 25 18 19 16 28 28 14 28 28 13 28 18 28 28 28 28 19 0 28 28 28 19 20 23 20 0 20 28 21 
0 21 28 0 21 28 22 28 28 0 

554
0 0 28 28 28 28 28 28 28 28 28 28 28 28 7 17 8 19 21 24 22 28 0 7 17 19 20 24 22 24 27 24 25 
0 24 19 20 12 28 0 8 20 19 20 22 25 24 28 8 20 19 20 25 24 28 20 25 27 26 19 20 22 19 28 15 10 
0 0 15 17 14 17 10 18 28 1 13 12 20 18 17 18 20 13 21 0 26 25 26 23 28 26 18 28 6 18 17 16 26 
0 25 26 28 28 28 28 28 28 25 26 6 26 0 26 23 25 26 21 25 28 28 16 13 26 25 23 17 20 28 28 28 18 
0 18 20 18 28 15 17 8 13 28 28 5 22 25 27 25 24 22 20 28 28 28 12 2 19 20 19 13 22 0 20 17 24 
0 22 20 25 27 25 20 22 24 28 28 28 20 19 20 19 22 13 20 17 24 22 20 25 27 25 20 22 24 28 28 28 20 
0 2 19 20 19 22 13 0 20 17 28 28 28 28 28 14 10 12 14 15 28 28 28 7 3 28 12 10 8 10 28 12 28 
0 13 20 27 25 21 23 21 16 18 20 28 28 28 16 0 15 16 15 18 28 16 13 0 27 24 25 20 23 21 16 18 14 
0 14 28 28 2 28 0 13 14 16 8 12 28 28 28 28 0 28 9 11 12 28 14 28 15 28 28 0 28 28 15 28 28 
0 15 28 17 28 15 28 28 28 3 15 28 28 28 28 17 0 15 28 28 28 0 15 28 28 0 24 28 13 24 28 28 22 
0 22 20 28 28 28 17 28 15 28 28 0 28 15 2

file name: ./data/chopin/nocturne_9_1_(c)unknown.mid
NAME nocturne_9_1_(c)unknown.mid
72
0 0 28 28 28 28 28 28 28 28 28 28 26 25 28 28 0 25 0 25 28 26 25 23 20 21 28 28 28 18 28 0 28 
0 0 28 28 26 25 26 25 0 27 25 23 21 28 20 25 24 21 28 28 28 18 28 0 28 28 28 28 28 28 28 28 28 
0 0 28 21 28 18 20 21 26 25 28 28 28 28 28 23 28 25 23 21 17 16 28 28 28 21 28 23 28 25 23 21 23 
0 25 28 28 28 28 28 0 28 28 28 28 26 25 28 28 0 25 0 25 28 26 25 23 20 21 28 28 28 18 28 0 28 
0 0 28 28 26 25 0 28 28 28 28 28 28 28 28 26 25 21 28 28 28 18 28 0 28 28 28 28 28 28 28 28 28 
0 22 28 25 28 28 23 26 28 25 28 28 28 24 28 26 24 25 27 0 28 28 28 28 28 28 26 21 28 28 28 20 0 
0 18 0 28 28 28 26 21 28 28 28 20 28 18 0 28 28 28 18 19 28 28 28 17 28 18 28 28 28 28 28 0 28 
0 0 28 28 28 25 28 28 28 28 0 24 28 28 28 28 0 23 25 23 0 20 0 18 0 16 28 28 0 18 0 20 28 
0 20 0 21 0 23 28 28 0 25 26 18 28 16 28 25 28 28 28 28 0 25 28 28 28 28 0 24 28 28 28 28 0 
0 22 24 22 0 17 0 19 28 28 28 28 0 22 24 22 0 17 0 18 